The code is based on the paper: Towards Simulating Foggy and Hazy Images and Evaluating Their Authenticity.
Referenced Github: https://github.com/noahzn/FoHIS

In [14]:
import cv2
import numpy as np
import math
import scipy.io as sio
import os
from PIL import Image
from scipy.special import gamma

In [15]:
BLOCK_SIZE_ROW     = 48
BLOCK_SIZE_COL     = 48
NORMALIZED_WIDTH   = 528
FEATURE_NUMBER     = 16
GRADIENT_THRESHOLD_L = 3
GRADIENT_THRESHOLD_R = 60
DARK_CHANNEL_THRESHOLD_L = 30
DARK_CHANNEL_THRESHOLD_R = 100

In [16]:
def filter2d(input_img, filter, frame):
    size = len(input_img), len(input_img[0])
    output = []
    for i in range(size[0]):
        temp = []
        for j in range(size[1]):
            temp.append(filter(input_img, (i, j), frame))
        output.append(temp)
    return output

In [17]:
def getDark(input_img, filter, frame):
    size = input_img.size
    output = []

    for x in range(size[1]):
        temp = []
        for y in range(size[0]):
            temp.append(min(input_img.getpixel((y, x))))

        output.append(temp)

    output = filter2d(output, filter, frame)

    output_img = Image.new('L', size)

    for x in range(size[1]):
        for y in range(size[0]):
            output_img.putpixel((y, x), output[x][y])

    return output_img

In [18]:
def minimizeFilter(input_img, point, size):
    begin = (point[0] - size[0] / 2, point[0] + size[0] / 2 + 1)
    end = (point[1] - size[1] / 2, point[1] + size[1] / 2 + 1)
    begin1, begin2 = int(begin[0]), int(begin[1])
    end1, end2 = int(end[0]), int(end[1])
    l = []
    for i in range(begin1, begin2):
        for j in range(end1, end2):
            if (i >= 0 and i < len(input_img)) and (j >= 0 and j < len(input_img[0])):
                l.append(input_img[i][j])
    return min(l)

In [19]:
def estimate_aggd_parameter(vec):
    vec = np.nan_to_num(vec)
    # print(vec)
    gam = [x/1000 for x in range(200, 10001, 1)]
    r_gam = [(gamma(2/x)**2/(gamma(1/x)*gamma(3/x))) for x in gam]
    leftstd = np.nan_to_num(np.sqrt(np.mean(vec[vec < 0]**2)))
    rightstd = np.nan_to_num(np.sqrt(np.mean(vec[vec > 0]**2)))
    gammahat = np.nan_to_num(leftstd / (rightstd+0.00001))
    rhat = np.nan_to_num((np.mean(np.abs(vec))**2) / np.nanmean(vec**2))
    rhatnorm = (rhat*(gammahat**3 + 1)*(gammahat + 1))/((gammahat**2 + 1)**2)
    m1 = (r_gam - rhatnorm)**2
    m2 = m1.tolist()
    array_position = m2.index(np.min(m1))
    alpha = gam[array_position]
    beta_l = leftstd * np.sqrt(gamma(1/alpha)/gamma(3/alpha))
    beta_r = rightstd * np.sqrt(gamma(1/alpha)/gamma(3/alpha))
    return alpha, beta_l, beta_r

In [20]:
def compute_features(gray, gradient):
    features = []
    for m in [gray]:
        μ = cv2.GaussianBlur(m, (5, 5), 5/6, borderType=cv2.BORDER_REPLICATE)
        σ = np.sqrt(abs(cv2.GaussianBlur(m*m, (5, 5), 5/6, borderType=cv2.BORDER_REPLICATE) - μ*μ))
        I = (m - μ) / (σ + 1)
        alpha, beta_l, beta_r = estimate_aggd_parameter(I)
        features.append(alpha)
        features.append((beta_l+beta_r)/2)
        I = np.log(I + 0.0001)
        shift1 = [(0, 1), (1, 0), (1, 1), (1, -1)]
        shift2 = [(-1, 0), (1, 0), (0, -1), (0, 1), (0, 0), (1, 1), (0, 1), (1, 0), (-1, -1), (1, 1), (-1, 1), (1, -1)]
        for i in shift1:
            D = np.roll(I, i, axis=(0, 1)) - I
            alpha, beta_l, beta_r = estimate_aggd_parameter(D)
            features.append(alpha)
            features.append((beta_l+beta_r)/2)

        for i in range(3):
            D = np.roll(I, shift2[4*i], axis=(0, 1)) + np.roll(I, shift2[4*i+1], axis=(0, 1)) \
                - np.roll(I, shift2[4*i+2], axis=(0, 1)) - np.roll(I, shift2[4*i+3], axis=(0, 1))
            alpha, beta_l, beta_r = estimate_aggd_parameter(D)

            features.append(alpha)
            features.append((beta_l+beta_r)/2)

    return features

In [21]:
def authenticity(img):
    data = sio.loadmat('Shift&Dense-Dataset-1.mat')
    mu_prisparam1 = data['mu1']
    mu_prisparam2 = data['mu2']
    cov_prisparam1 = data['cov1']
    cov_prisparam2 = data['cov2']

    img = cv2.resize(cv2.imread(img), (NORMALIZED_WIDTH, NORMALIZED_WIDTH), interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    block_rownum = math.floor(gray.shape[0] / BLOCK_SIZE_ROW)
    block_colnum = math.floor(gray.shape[1] / BLOCK_SIZE_COL)
    img = img[:block_rownum * BLOCK_SIZE_ROW, :block_colnum * BLOCK_SIZE_COL, :]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)[:, :, 0]

    # gradient magnitude
    gradx = cv2.Sobel(gray, cv2.CV_16S, 1, 0, ksize=3)
    grady = cv2.Sobel(gray, cv2.CV_16S, 0, 1, ksize=3)
    absX = cv2.convertScaleAbs(gradx)
    absY = cv2.convertScaleAbs(grady)
    gradient = cv2.addWeighted(absX, 0.5, absY, 0.5, 0)
    gradient2 = gradient
    gradient2[gradient2 < 20] = 0
    gradient2[gradient2 >= 20] = 255

    # dark channel
    dark_image = np.asarray(getDark(Image.fromarray(np.uint8(img)), minimizeFilter, (10, 10)))

    quality = []
    features1_list_all = []
    features2_list_all = []
    for i in range(block_rownum):
        for j in range(block_colnum):
            features1_list = []
            features2_list = []
            crop_row_start = i * BLOCK_SIZE_ROW
            crop_row_end = (i + 1) * BLOCK_SIZE_ROW
            crop_col_start = j * BLOCK_SIZE_COL
            crop_col_end = (j + 1) * BLOCK_SIZE_COL

            crop_gray = gray[crop_row_start: crop_row_end, crop_col_start:crop_col_end]
            crop_img = img[crop_row_start: crop_row_end, crop_col_start:crop_col_end]
            crop_gradient = gradient[crop_row_start: crop_row_end, crop_col_start:crop_col_end]
            crop_gradient2 = gradient2[crop_row_start: crop_row_end, crop_col_start:crop_col_end]
            crop_dark_image = dark_image[crop_row_start: crop_row_end, crop_col_start:crop_col_end]

            if np.mean(crop_dark_image) < DARK_CHANNEL_THRESHOLD_L:
                if np.count_nonzero(crop_gradient2) > 400:
                    features1_list.extend(
                        compute_features(crop_gray.astype(np.float64), crop_gradient.astype(np.float64)))
                    cv2.rectangle(img, (crop_col_start, crop_row_start), (crop_col_end, crop_row_end),
                                  (0, 255, 0))
                else:
                    features1_list.extend(
                        compute_features(crop_gray.astype(np.float64), crop_gradient.astype(np.float64)))
                    cv2.rectangle(img, (crop_col_start, crop_row_start), (crop_col_end, crop_row_end),
                                  (255, 0, 0))

            elif np.mean(crop_dark_image) >= DARK_CHANNEL_THRESHOLD_L:
                features2_list.extend(
                    compute_features(crop_gray.astype(np.float64), crop_gradient.astype(np.float64)))
                cv2.rectangle(img, (crop_col_start, crop_row_start), (crop_col_end, crop_row_end),
                              (255, 0, 255))

            features1_list_all.extend(features1_list)
            features2_list_all.extend(features2_list)

    if len(features1_list_all) != 0:
        features1 = np.array(features1_list_all).reshape((int(len(features1_list_all) / FEATURE_NUMBER)), FEATURE_NUMBER)
        if features1.shape[0] > 1:
            mu_distparam1 = (np.mean(features1, axis=0))
            cov_distparam1 = np.cov(features1.reshape(features1.shape[1], features1.shape[0]))
            invcov_param1 = np.linalg.inv((cov_prisparam1 + cov_distparam1) / 2)
            q1 = np.sqrt(
                np.dot(np.dot((mu_prisparam1 - mu_distparam1), invcov_param1), np.transpose(mu_prisparam1 - mu_distparam1)))
            quality.append(np.nanmean(q1))
        else:
            features2_list_all.extend(features2_list_all)

    if len(features2_list_all) != 0:
        features2 = np.array(features2_list_all).reshape((int(len(features2_list_all) / FEATURE_NUMBER)), FEATURE_NUMBER)
        # input(features2)
        mu_distparam2 = (np.mean(features2, axis=0))
        cov_distparam2 = np.cov(features2.reshape(features2.shape[1], features2.shape[0]))

        # input(mu_distparam2)
        invcov_param2 = np.linalg.inv((cov_prisparam2 + cov_distparam2) / 2)
        q2 = np.sqrt(
            np.dot(np.dot((mu_prisparam2 - mu_distparam2), invcov_param2), np.transpose(mu_prisparam2 - mu_distparam2)))

        # input(q2)
        quality.append(np.nanmean(q2))
    return quality


In [22]:
authenticity('../Fog_Simulator-FoHIS/Results/Shift_Images/image_0c3b-b343_00000000_img_front.jpg')

/var/folders/vw/c2j4rkjd52z6m0m84r33nr7r0000gn/T/ipykernel_19333/3702971359.py:10: RuntimeWarning: invalid value encountered in log
  I = np.log(I + 0.0001)


[2.990390269819463, 5.97826601781121]

In [ ]:
def process_images_in_folder(folder_path):
    results = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            result = authenticity(image_path)
            results.append((image_path, result))
    return results

folders = ['../Fog_Simulator-FoHIS/Results/With_Depth_Maps', '../Fog_Simulator-FoHIS/Results/With_Depth-Anything', '../Fog_Simulator-FoHIS/Results/With_MiDaS']
all_results = []

for folder in folders:
    folder_results = process_images_in_folder(folder)
    all_results.extend(folder_results)

# Save the results
with open('results.txt', 'w') as f:
    for image_path, result in all_results:
        f.write(f"{image_path}: {result}\n")

/var/folders/vw/c2j4rkjd52z6m0m84r33nr7r0000gn/T/ipykernel_19333/3702971359.py:10: RuntimeWarning: invalid value encountered in log
  I = np.log(I + 0.0001)


In [ ]:
all_results